In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import HTML
from celluloid import Camera
from matplotlib import animation
import matplotlib.pyplot as plt
import lib.helpers as hlp
import lib.backtest as backtest
import lib.plotting as plotting
import lib.kelly as kelly
import lib.random_path_generation as rpg
import lib.convolution as convolution
import lib.black_scholes as bs

In [ ]:
# all backtest settings
historical_df = hlp.get_historical_prices_df()
price_sequence = historical_df['Ethereum'].dropna()
returns_sequence = price_sequence.pct_change()[1:]
simulation_length_days = 720 # number of option life cycles to use for backtest
duration = 7 # number of the expiration days for put option
# simulation_length_cycles = int(simulation_length_days / duration)
premium_offset = 0. # 0.1 = 10% additional relative premium
strike = 1.1 # 1.1 = 10% in the money
number_of_paths = 200 # number of the random paths to do backtest
bonding_curve_resolution = 10 # number of utils on the [0, 1] segment, passed to the get_kelly_curve
convolution_n_paths = 1000
initial_util = 0.66
option_pricing_method = 'kelly' # could be 'kelly' or 'bs'
option_type = 'call'
# n_bins = 'auto'


In [ ]:
# Create Utils list
##########################################################################
utils = rpg.generate_utils_list(simulation_length_days, 
                                initial_util=initial_util, 
                                duration=duration, randomize=False)
##########################################################################

# Returns Paths
##########################################################################
underlying_n_daily_returns = convolution.daily_returns_to_n_day_returns(returns_sequence, 
                                                                        number_of_paths=convolution_n_paths, 
                                                                        n_days=duration)
random_return_paths_cycles_df = rpg.generate_returns_paths_from_returns(
    underlying_n_daily_returns, number_of_paths, simulation_length_days//duration, output_as_df=True
)
##########################################################################

# Premiums Calculation
##########################################################################
if option_pricing_method == 'kelly':
    premiums, premium_vs_util_df, premium_vs_util_fit_params = kelly.get_premiums_list_with_all_calculations(
        utils,
        premium_offset=premium_offset,
        fit_params=None,
        bonding_curve_resolution=bonding_curve_resolution,
        underlying_n_daily_returns=underlying_n_daily_returns,
        strike=strike,
        option_type=option_type
    )
elif option_pricing_method == 'bs':
    if option_type=='put':
        premium = bs.calculate_bs_put_premium(
            yearly_returns_std=np.std(returns_sequence)*np.sqrt(365),
            years_to_expiry=duration/365,
            strike_price=strike,
            spot_price=1,
            risk_free_interest_rate=0.01
        )
    else:
        premium = bs.calculate_bs_call_premium(
            yearly_returns_std=np.std(returns_sequence)*np.sqrt(365),
            years_to_expiry=duration/365,
            strike_price=strike,
            spot_price=1,
            risk_free_interest_rate=0.01
        )
    premiums = hlp.extend_list_to_be_daily([premium] * (simulation_length_days // duration), 
                                           duration, simulation_length_days)
    premium_vs_util_df = pd.DataFrame({'util': np.linspace(0, 1, bonding_curve_resolution), 
                                       'premium': [premium] * bonding_curve_resolution})
    premium_vs_util_fit_params = [0, 0, 0, premium]
    
    
else:
    raise ValueError(f'Unknown option_pricing_method: {option_pricing_method}')
# Do Actual Backtest
##########################################################################
bankroll_df = backtest.run_backtest(
    random_return_paths_cycles_df,
    utils,
    premiums,
    strike,
    duration, 
    option_type=option_type
)
##########################################################################

In [ ]:
# plot backtest results
plotting.plot_kelly_curve(premium_vs_util_df)

In [ ]:
plotting.plot_bankroll_df(bankroll_df)

In [ ]:
kde_df = pd.DataFrame()

for col in bankroll_df:
    max_drawdown = backtest.calculate_max_drawdown(bankroll_df[col])
    cagr = backtest.calculate_cagr(bankroll_df[col])
    kde_df = kde_df.append({'cagr': cagr, 
                            'max_drawdown': max_drawdown}, 
                           ignore_index=True)

sns.kdeplot(data=kde_df, x = 'max_drawdown', y='cagr', 
            color='b', shade=True, cmap='Blues', alpha=0.6)


In [ ]:
plt.hist(kde_df['cagr'])

In [ ]:
np.median(kde_df['cagr'])

In [ ]:
# kde evolution with path length growth
fig = plt.figure()
camera = Camera(fig)
median_drawdown_list = []
path_len_list = [180 * i for i in range(1, 10)]

for simulated_price_path_len in path_len_list:
    print(simulated_price_path_len)
    # Create Utils list
    ##########################################################################
    utils = rpg.generate_utils_list(simulated_price_path_len, 
                                    initial_util=initial_util, 
                                    duration=duration, randomize=False)
    ##########################################################################

    # Returns Paths
    ##########################################################################
    underlying_n_daily_returns = convolution.daily_returns_to_n_day_returns(returns_sequence, 
                                                                            number_of_paths=convolution_n_paths, 
                                                                            n_days=duration)
    random_return_paths_cycles_df = rpg.generate_returns_paths_from_returns(
        underlying_n_daily_returns, number_of_paths, simulated_price_path_len//duration, output_as_df=True
    )
    ##########################################################################

    # Premiums Calculation
    ##########################################################################
    if option_pricing_method == 'kelly':
        premiums, premium_vs_util_df, premium_vs_util_fit_params = kelly.get_premiums_list_with_all_calculations(
            utils,
            premium_offset=premium_offset,
            fit_params=None,
            bonding_curve_resolution=bonding_curve_resolution,
            underlying_n_daily_returns=underlying_n_daily_returns,
            strike=strike,
            option_type=option_type
        )
    elif option_pricing_method == 'bs':
        if option_type=='put':
            premium = bs.calculate_bs_put_premium(
                yearly_returns_std=np.std(returns_sequence)*np.sqrt(365),
                years_to_expiry=duration/365,
                strike_price=strike,
                spot_price=1,
                risk_free_interest_rate=0.01
            )
        else:
            premium = bs.calculate_bs_call_premium(
                yearly_returns_std=np.std(returns_sequence)*np.sqrt(365),
                years_to_expiry=duration/365,
                strike_price=strike,
                spot_price=1,
                risk_free_interest_rate=0.01
            )
        premiums = hlp.extend_list_to_be_daily([premium] * (simulated_price_path_len // duration), 
                                            duration, simulation_length_days)
        premium_vs_util_df = pd.DataFrame({'util': np.linspace(0, 1, bonding_curve_resolution), 
                                        'premium': [premium] * bonding_curve_resolution})
        premium_vs_util_fit_params = [0, 0, 0, premium]
        
        
    else:
        raise ValueError(f'Unknown option_pricing_method: {option_pricing_method}')
    # Do Actual Backtest
    ##########################################################################
    bankroll_df = backtest.run_backtest(
        random_return_paths_cycles_df,
        utils,
        premiums,
        strike,
        duration,
        option_type=option_type
    )
    ##########################################################################
    # kde of bankrolls vs drawdowns
    kde_df = pd.DataFrame()

    for col in bankroll_df:
        max_drawdown = backtest.calculate_max_drawdown(bankroll_df[col])
        cagr = backtest.calculate_cagr(bankroll_df[col])
        kde_df = kde_df.append({'cagr': cagr, 
                                'max_drawdown': max_drawdown}, 
                            ignore_index=True)

    sns.kdeplot(data=kde_df, x = 'max_drawdown', y='cagr', 
                color='b', shade=True, cmap='Blues', alpha=0.6)
    plt.tight_layout()
    plt.legend([f'simulated_price_path_len={simulated_price_path_len}'])
    camera.snap()
    
animation = camera.animate(interval=500)
# animation.save(f'{hlp.MEDIA_DIR}movie.gif')
HTML(animation.to_html5_video())

In [ ]:
# kde plots for different strikes
fig, ax = plt.subplots()
camera = Camera(fig)

for strike in [0.9, 0.95, 1, 1.05, 1.1]:
    print(f'{strike=}')
    # Create Utils list
    ##########################################################################
    utils = rpg.generate_utils_list(simulation_length_days, 
                                         initial_util=initial_util, 
                                         duration=duration, randomize=False)
    ##########################################################################

    # Returns Paths
    ##########################################################################
    underlying_n_daily_returns = convolution.daily_returns_to_n_day_returns(returns_sequence, 
                                                                            number_of_paths=convolution_n_paths, 
                                                                            n_days=duration)
    random_return_paths_cycles_df = rpg.generate_returns_paths_from_returns(
        underlying_n_daily_returns, number_of_paths, simulation_length_days//duration, output_as_df=True
    )
    ##########################################################################

    # Premiums Calculation
    ##########################################################################
    if option_pricing_method == 'kelly':
        premiums, premium_vs_util_df, premium_vs_util_fit_params = kelly.get_premiums_list_with_all_calculations(
            utils,
            premium_offset=premium_offset,
            fit_params=None,
            bonding_curve_resolution=bonding_curve_resolution,
            underlying_n_daily_returns=underlying_n_daily_returns,
            strike=strike,
            option_type=option_type
        )
    elif option_pricing_method == 'bs':
        if option_type=='put':
            premium = bs.calculate_bs_put_premium(
                yearly_returns_std=np.std(returns_sequence)*np.sqrt(365),
                years_to_expiry=duration/365,
                strike_price=strike,
                spot_price=1,
                risk_free_interest_rate=0.01
            )
        else:
            premium = bs.calculate_bs_call_premium(
                yearly_returns_std=np.std(returns_sequence)*np.sqrt(365),
                years_to_expiry=duration/365,
                strike_price=strike,
                spot_price=1,
                risk_free_interest_rate=0.01
            )
        premiums = hlp.extend_list_to_be_daily([premium] * (simulation_length_days // duration), 
                                            duration, simulation_length_days)
        premium_vs_util_df = pd.DataFrame({'util': np.linspace(0, 1, bonding_curve_resolution), 
                                        'premium': [premium] * bonding_curve_resolution})
        premium_vs_util_fit_params = [0, 0, 0, premium]
        
        
    else:
        raise ValueError(f'Unknown option_pricing_method: {option_pricing_method}')
    # Do Actual Backtest
    ##########################################################################
    bankroll_df = backtest.run_backtest(
        random_return_paths_cycles_df,
        utils,
        premiums,
        strike,
        duration,
        option_type=option_type
    )
    ##########################################################################
    # kde of bankrolls vs drawdowns
    kde_df = pd.DataFrame()

    for col in bankroll_df:
        max_drawdown = backtest.calculate_max_drawdown(bankroll_df[col])
        cagr = backtest.calculate_cagr(bankroll_df[col])
        kde_df = kde_df.append({'cagr': cagr, 
                                'max_drawdown': max_drawdown}, 
                            ignore_index=True)

    sns.kdeplot(data=kde_df, x = 'max_drawdown', y='cagr', 
                color='b', shade=True, cmap='Blues', alpha=0.6)
    plt.tight_layout()
    plt.legend([f'strike={strike}'])
    camera.snap()
    
animation = camera.animate(interval=500)
# animation.save(f'{hlp.MEDIA_DIR}movie.gif')
HTML(animation.to_html5_video())


In [ ]:
# kde plots for different durations
fig, ax = plt.subplots()
camera = Camera(fig)

for duration in range(1, 30, 5):
    # Create Utils list
    ##########################################################################
    utils = rpg.generate_utils_list(simulation_length_days, 
                                         initial_util=initial_util, 
                                         duration=duration, randomize=False)
    ##########################################################################

    # Returns Paths
    ##########################################################################
    underlying_n_daily_returns = convolution.daily_returns_to_n_day_returns(returns_sequence, 
                                                                            number_of_paths=convolution_n_paths, 
                                                                            n_days=duration)
    random_return_paths_cycles_df = rpg.generate_returns_paths_from_returns(
        underlying_n_daily_returns, number_of_paths, simulation_length_days//duration, output_as_df=True
    )
    ##########################################################################

    # Premiums Calculation
    ##########################################################################
    if option_pricing_method == 'kelly':
        premiums, premium_vs_util_df, premium_vs_util_fit_params = kelly.get_premiums_list_with_all_calculations(
            utils,
            premium_offset=premium_offset,
            fit_params=None,
            bonding_curve_resolution=bonding_curve_resolution,
            underlying_n_daily_returns=underlying_n_daily_returns,
            strike=strike,
            option_type=option_type
        )
    elif option_pricing_method == 'bs':
        if option_type=='put':
            premium = bs.calculate_bs_put_premium(
                yearly_returns_std=np.std(returns_sequence)*np.sqrt(365),
                years_to_expiry=duration/365,
                strike_price=strike,
                spot_price=1,
                risk_free_interest_rate=0.01
            )
        else:
            premium = bs.calculate_bs_call_premium(
                yearly_returns_std=np.std(returns_sequence)*np.sqrt(365),
                years_to_expiry=duration/365,
                strike_price=strike,
                spot_price=1,
                risk_free_interest_rate=0.01
            )
        premiums = hlp.extend_list_to_be_daily([premium] * (simulation_length_days // duration), 
                                            duration, simulation_length_days)
        premium_vs_util_df = pd.DataFrame({'util': np.linspace(0, 1, bonding_curve_resolution), 
                                        'premium': [premium] * bonding_curve_resolution})
        premium_vs_util_fit_params = [0, 0, 0, premium]
        
        
    else:
        raise ValueError(f'Unknown option_pricing_method: {option_pricing_method}')
    # Do Actual Backtest
    ##########################################################################
    bankroll_df = backtest.run_backtest(
        random_return_paths_cycles_df,
        utils,
        premiums,
        strike,
        duration,
        option_type=option_type
    )
    ##########################################################################
    # kde of bankrolls vs drawdowns
    kde_df = pd.DataFrame()

    for col in bankroll_df:
        max_drawdown = backtest.calculate_max_drawdown(bankroll_df[col])
        cagr = backtest.calculate_cagr(bankroll_df[col])
        kde_df = kde_df.append({'cagr': cagr, 
                                'max_drawdown': max_drawdown}, 
                            ignore_index=True)

    sns.kdeplot(data=kde_df, x = 'max_drawdown', y='cagr', 
                color='b', shade=True, cmap='Blues', alpha=0.6)
    plt.tight_layout()
    plt.legend([f'duration={duration}'])
    camera.snap()
    
animation = camera.animate(interval=500)
# animation.save(f'{hlp.MEDIA_DIR}movie.gif')
HTML(animation.to_html5_video())